In [12]:
import os
from glob import glob

import numpy as np
import string
from PIL import Image
import pytesseract

import cv2
import torch

# git clone https://github.com/ultralytics/yolov3
# yolov3 folder 내 존재
# from utils.general import non_max_suppression
# from utils.torch_utils import select_device
# from models.common import DetectMultiBackend

#### 모자이크 함수

In [3]:
def mosaic(src, ratio=0.08):
    small = cv2.resize(src, None, fx=ratio, fy=ratio, interpolation=cv2.INTER_NEAREST)
    return cv2.resize(small, src.shape[:2][::-1], interpolation=cv2.INTER_NEAREST)

In [4]:
def mosaic_area(src, area, ratio=0.08):
    for i in area:
        x = i[0]
        y = i[1]
        width = i[2]
        height = i[3]

        # 모자이크 수행
        src[y:y + height, x:x + width] = mosaic(src[y:y + height, x:x + width], ratio)

    return src

#### 블러 함수

In [15]:
def blur(src):
    ksize = 30
    roi = cv2.blur(src, (ksize, ksize))
    return roi

In [16]:
def blur_area(src, area, ratio=0.08):
    for i in area:
        x = i[0]
        y = i[1]
        width = i[2]
        height = i[3]

        src[y:y + height, x:x + width] = blur(src[y:y + height, x:x + width])

    return src

### text용 draw bbox

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r"Tesseract-OCR/tesseract.exe"

In [ ]:
# conf = r'-c tessedit_char_whitelist='+string.ascii_letters
conf = r"-c tessedit_char_whitelist=" + string.digits
conf = conf + string.ascii_letters

In [ ]:
def draw_boxes_on_text(img):
    raw_data = pytesseract.image_to_data(img)

    print(raw_data)
    for count, data in enumerate(raw_data.splitlines()):
        if count > 0:
            data = data.split()
            if len(data) == 12:
                x, y, w, h, content = int(data[6]), int(data[7]), int(data[8]), int(data[9]), data[11]
                cv2.rectangle(img, (x, y), (w + x, h + y), (255, 255, 255), 2)

                roi = img[y : y + h, x : x + w]
                roi = cv2.blur(roi, (30, 30))
                img[y : y + h, x : x + w] = roi

                # cv2.putText(img, content, (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255) , 1)
    return img

### torch 연결 확인

In [ ]:
images = [
    "Test/img95.jpg",
    "Test/img391.jpg",
    "Test/img561.jpg"
]

In [11]:
torch.cuda.is_available()

True

In [10]:
face_model = torch.hub.load("ultralytics/yolov3", "yolov3")

Using cache found in C:\Users\User/.cache\torch\hub\ultralytics_yolov3_master
YOLOv3  2022-5-5 torch 1.7.1+cu110 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)

Fusing layers... 
Model Summary: 261 layers, 61922845 parameters, 0 gradients, 156.1 GFLOPs
Adding AutoShape... 


In [ ]:
model_path = "final/exp/weights/best.pt"
# text_model = torch.load(model_path)
device = select_device(0)
text_model = DetectMultiBackend(model_path, device=device)

img = cv2.imread(images[0])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = np.transpose(img, (2, 0, 1))
img = img.reshape(-1, img.shape[0], img.shape[1], img.shape[2])
img = np.divide(img, 255)
pred = text_model(torch.from_numpy(img))

pred = text_model(images[0])

In [37]:
pred = face_model(images[0])

In [18]:
pred.xyxy

[tensor([[ 99.46055, 212.78828, 250.88907, 332.35471,   0.91650,   0.00000],
         [ 23.37891,   0.77500, 325.43439, 181.12656,   0.44824,  45.00000],
         [ 19.90547,   1.44297, 319.02188, 180.99297,   0.35010,  74.00000]], device='cuda:0')]

In [19]:
pred.xywh

[tensor([[175.17480, 272.57150, 151.42853, 119.56642,   0.91650,   0.00000],
         [174.40665,  90.95078, 302.05548, 180.35156,   0.44824,  45.00000],
         [169.46367,  91.21796, 299.11642, 179.55000,   0.35010,  74.00000]], device='cuda:0')]

In [38]:
areas = []
for per_img in pred.xyxy:
    for x, y, x2, y2, _, _ in per_img:
        x, y, w, h = map(int, [x, y, x2 - x, y2 - y])
        areas.append([x, y, w, h])

In [45]:
frame = cv2.imread(images[0])
output = blur_area(frame, areas)
# frame = mosaic_area(frame, areas)

output = cv2.resize(frame, (frame.shape[0], frame.shape[1]))
cv2.imshow("test", output)
cv2.waitKey(0)
cv2.destroyAllWindows()

### face detect & blur

In [13]:
torch.cuda.is_available()

True

In [22]:
# images = glob("test/faces/images/*")
# images = ["test/faces/origin/5.jpg", "test/faces/origin/2.jpg"]
images = [
    "Test/img95.jpg",
    "Test/img391.jpg",
    "Test/img561.jpg"
]
# outputs = "test/blur"
# assert os.path.exists(outputs)

In [17]:
face_model = torch.hub.load("ultralytics/yolov3", "yolov3")

Using cache found in C:\Users\User/.cache\torch\hub\ultralytics_yolov3_master
YOLOv3  2022-5-5 torch 1.7.1+cu110 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)

Fusing layers... 
Model Summary: 261 layers, 61922845 parameters, 0 gradients, 156.1 GFLOPs
Adding AutoShape... 


In [27]:
for img in images:
    # print(img)
    pred = face_model(img)
    frame = cv2.imread(img)

    areas = []
    for per_img in pred.xyxy:
        for x, y, x2, y2, _, c in per_img:
            # if int(c) != 0:
            #     continue
            x, y, w, h = map(int, [x, y, x2 - x, y2 - y])
            # frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 3)
            areas.append([x, y, w, h])

    # cv2.imshow("test", frame)

    output = blur_area(frame, areas)
    cv2.imshow("test", output)
    # frame = mosaic_area(frame, areas)

    # output = cv2.resize(frame, (frame.shape[0], frame.shape[1]))
    # cv2.imshow("test", output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # output_path = os.path.join(outputs, img.rsplit("\\", 1)[1])
    output_path = img.rsplit("/", 1)[1]
    # cv2.imwrite(output_path, frame)
    cv2.imwrite(output_path, output)
print("finished!!")

finished!!


### image blur

In [10]:
# images = glob("test/images/*")
images = ["test/images/C.jpg"]
texts = glob("test/texts/preds/*.txt")
outputs = "test/blur"
assert os.path.exists(outputs)

In [11]:
for idx, img_address in enumerate(images):
    # print(img_address)
    text_address = ""
    for t in texts:
        # if img_address.rsplit("\\", 1)[1].rsplit(".", 1)[0] == t.rsplit("\\", 1)[1].rsplit(".", 1)[0]:
        if img_address.rsplit("/", 1)[1].rsplit(".", 1)[0] == t.rsplit("\\", 1)[1].rsplit(".", 1)[0]:
            text_address = t
            break
    # print(text_address)
    if len(text_address) < 1:
        print("뭔가 이상해, pred를 가지고 한 건데 pred text가 없대")
        continue

    frame = cv2.imread(img_address)
    # print(img_address, frame.shape)
    # cv2.imshow("origin", frame)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # use text detect result
    areas = []
    with open(text_address, "r") as tf:
        lines = tf.readlines()
    for line in lines:
        parts = line.rsplit("\n", 1)[0].split(" ")
        # print(parts)
        cx, cy, w, h = map(float, parts[1:])
        x, y, w, h = map(int, [(cx - w / 2) * frame.shape[1], (cy - h / 2) * frame.shape[0], (cx + w / 2) * frame.shape[1], (cy + h / 2) * frame.shape[0]])
        # x, y, w, h = map(int, [cx * frame.shape[0], cy * frame.shape[1], w * frame.shape[0], (cy - h / 2) * frame.shape[1]])

        # x = max(min(x, frame.shape[0]), 0)
        # y = max(min(y, frame.shape[1]), 0)
        # if x + w > frame.shape[0]:
        #     w = frame.shape[0] - x
        # if y + h > frame.shape[1]:
        #     y = frame.shape[1] - y

        frame = cv2.rectangle(frame, (x, y), (w + h, y + h), (255, 0, 0), 3)
        # areas.append([x, y, w, h])
    # print(areas)

    # output = blur_area(frame, areas)
    # frame = mosaic_area(frame, areas)
    # output = cv2.resize(output, (500, 500))
    # output = cv2.resize(output, (frame.shape[0], frame.shape[1]))
    # cv2.imshow("test", output)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # output_path = os.path.join(outputs, img_address.rsplit("\\", 1)[1])
    output_path = os.path.join(outputs, img_address.rsplit("/", 1)[1])
    # cv2.imwrite(output_path, output)
    cv2.imwrite(output_path, frame)
print("finished!!")

finished!!


### total

In [5]:
images = glob("test/images/*")
# images = ["images/F.jpg", "images/5.jpg"]
# images = ["test/images/F.jpg", "test/images/A.jpg"]
# images = [
#     "Test/img95.jpg",
#     "Test/img391.jpg",
#     "Test/img561.jpg"
# ]
texts = glob("test/preds/*.txt")

In [6]:
face_model = torch.hub.load("ultralytics/yolov3", "yolov3")

Using cache found in C:\Users\User/.cache\torch\hub\ultralytics_yolov3_master
YOLOv3  2022-5-5 torch 1.7.1+cu110 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)

Fusing layers... 
Model Summary: 261 layers, 61922845 parameters, 0 gradients, 156.1 GFLOPs
Adding AutoShape... 


In [7]:
for img_address in images:
    # print(img_address)
    text_address = ""
    for t in texts:
        if img_address.rsplit("\\", 1)[1].rsplit(".", 1)[0] == t.rsplit("\\", 1)[1].rsplit(".", 1)[0]:
            text_address = t
            break
    # print(text_address)
    if len(text_address) < 1:
        print("뭔가 이상해, pred를 가지고 한 건데 pred text가 없대")
        break

    areas = []
    # * face detection
    pred = face_model(img_address)
    for per_img in pred.xyxy:
        for x, y, x2, y2, _, c in per_img:
            if int(c) != 0:
                continue
            x, y, w, h = map(int, [x, y, x2 - x, y2 - y])
            areas.append([x, y, w, h])

    frame = cv2.imread(img_address)
    # cv2.imshow("origin", frame)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # * use text detect result
    with open(text_address, "r") as tf:
        lines = tf.readlines()
    for line in lines:
        parts = line.rsplit("\n", 1)[0].split(" ")
        cx, cy, w, h = map(float, parts[1:5])
        x, y, w, h = map(int, [(cx - w / 2) * frame.shape[0], (cy - h / 2) * frame.shape[1], (cx + w / 2) * frame.shape[0], (cy + h / 2) * frame.shape[1]])

        # x, y, w, h = map(int, [cx * frame.shape[0], cy * frame.shape[1], w * frame.shape[0], (cy - h / 2) * frame.shape[1]])

        # ? draw (test)
        frame = cv2.rectangle(frame, (x, y), (w + h, y + h), (255, 0, 0), 3)
        # ? save
        # areas.append([x, y, w, h])
    # print(areas)

    # ? draw (test)
    texts = pytesseract.image_to_string(frame)
    temp = draw_boxes_on_text(frame)

    # ? save
    # raw_data = pytesseract.image_to_data(frame)
    # for count, data in enumerate(raw_data.splitlines()):
    #     if count > 0:
    #         data = data.split()
    #         if len(data) == 12:
    #             x, y, w, h, content = int(data[6]), int(data[7]), int(data[8]), int(data[9]), data[11]
    #             areas.append([x, y, w, h])

    output = blur_area(temp, areas)
    # frame = mosaic_area(frame, areas)

    output = cv2.resize(output, (500, 500))
    # output = cv2.resize(output, (frame.shape[0], frame.shape[1]))
    cv2.imshow("output", output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()